# Simulation #2
Having written the functions required to implement estimation of $L$, we now experiment with a few different model parameters. First, however, it will be helpful to wrap all of the helper functions into a single ```estimate_L``` function.

In [1]:
include("functions.jl");
include("nnlsq.jl");

In [2]:
?estimate_L

search: estimate_L estimate SkewSymmetricMatrixSpace SkewSymmetricMatrixShape



```
estimate_L(state_space, sample_size, S, φ, ϵ, basis_grid_size, integral_resolution; dist)
```

Combines all of the helper functions into a single API for estimating the matrix L.

Returns L.


In [3]:
state_space = Dict("min"=>0, "max"=>2π, "grid_resolution"=>1000);

## Varying the integral resolution
I don't expect that varying the resolution of the integral calculations should have a significant effect.

In [17]:
integral_resolutions = [100, 200, 500, 1000];
results = [];
for res in integral_resolutions
    L = estimate_L(state_space, 1000, S, φ, 1., 10, res; dist="normal");
    push!(results, L);
end

Academic license - for non-commercial use only - expires 2021-08-05
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 100 rows, 1100 columns and 97681 nonzeros
Model fingerprint: 0xf7e80c4e
Model has 100 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-13, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+00]
Presolve time: 0.02s
Presolved: 100 rows, 1100 columns, 97681 nonzeros
Presolved model has 100 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 4.950e+03
 Factor NZ  : 5.050e+03 (roughly 1 MByte of memory)
 Factor Ops : 3.384e+05 (less than 1 second per iteration)
 Threads    : 2

                  Objective                Residual
Iter       Primal          D

Objective value is: 81.24474182984375


In [29]:
for i in 1:4
    resolution = integral_resolutions[i];
    on = opnorm(results[i]);
    println("With resolution $resolution, opnorm is $on")
end

With resolution 100, opnorm is 32.289271094558636
With resolution 200, opnorm is 128.60411858547178
With resolution 500, opnorm is 737.306420092123
With resolution 1000, opnorm is 2818.855210018158


## Varying sample size
Similarly, sample size should not have much of an effect.

In [31]:
sample_sizes = [100, 1000, 10000, 100000];
results = [];
for sample_size in sample_sizes
    L = estimate_L(state_space, sample_size, S, φ, 1., 10, 100; dist="normal");
    push!(results, L);
end

Academic license - for non-commercial use only - expires 2021-08-05
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 100 rows, 200 columns and 9842 nonzeros
Model fingerprint: 0xfc3d4982
Model has 100 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-13, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+00]
Presolve time: 0.00s
Presolved: 100 rows, 200 columns, 9842 nonzeros
Presolved model has 100 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 4.950e+03
 Factor NZ  : 5.050e+03
 Factor Ops : 3.384e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     C

Objective value is: 1.034450382803925


In [32]:
for i in 1:4
    sample_size = sample_sizes[i];
    on = opnorm(results[i]);
    println("With sample size $sample_size, opnorm is $on")
end

With sample size 100, opnorm is 300.9230971334263
With sample size 1000, opnorm is 27.941903079186044
With sample size 10000, opnorm is 2.9342675943364376
With sample size 100000, opnorm is 0.30018303065931123
